In [1]:
import numpy as np
from collections import defaultdict, Iterator
from numpy.lib.recfunctions import merge_arrays
from pprint import pprint

In [2]:
class Delayed: 
    pass

In [3]:
class DelayedArray(np.ndarray): 
    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        
        if len(obj.shape) == 1:
            obj = obj.reshape((-1, 1))
        
        return obj

In [4]:
class NpStore: 
    def __init__(self): 
        self._pages = defaultdict(lambda: None)
        self._delay_buffer = defaultdict(lambda: None)

    @staticmethod
    def sane_append(array1, array2): 
        if array1 is None:
            return array2
        return np.append(array1, array2)
    
    @staticmethod
    def get_shapes(values, flatten=False):
        shapes = [v.shape if hasattr(v, "shape") else tuple() for v in values]  
        
        if flatten:
            if len(set([shp[0] for shp in shapes])) > 1: 
                raise ValueError("When flattening, the first dimension of all values must be equal")
            
            shapes = [shp[1:] for shp in shapes]
        
        return shapes
    
    def dict_to_array(self, dictionary, flatten=False, value_parser=np.atleast_1d): 
        
        units = [d["unit"] for d in dictionary.values()]
        field_names = ["{} [{}]".format(*nu) for nu in zip(dictionary.keys(), units)]

        values = [value_parser(d["value"]) for d in dictionary.values()]
        shapes = self.get_shapes(values, flatten=flatten)

        types = [str(shp) + str(i.dtype) for shp, i in zip(shapes, values)]
        dtype = list(zip(field_names, types))

        if flatten: 
            rval= np.array([tuple(i) for i in zip(*values)], dtype=dtype)
        else:
            rval = np.array([tuple(values)], dtype=dtype)

        return rval 
    
    def add(self, record):
        
        independents = [(name, value) for name, value in record.items() if value.get('independent_parameter', False)]
        independents = self.dict_to_array(dict(independents))
        
        dependents = [(name, value) for name, value in record.items() if not value.get('independent_parameter', False)]
        
        for name, parameter_record in dependents: 
            
            dependent = dict([(name, parameter_record)])
            
            if isinstance(parameter_record["value"], Delayed): 
                self._delay_buffer[name] = self.sane_append(self._delay_buffer[name], independents)
                continue
                
            elif isinstance(parameter_record["value"], DelayedArray):
                independents = np.append(self._delay_buffer[name], independents)
                dependent = self.dict_to_array(dependent, flatten=True)
                del self._delay_buffer[name]
                
            else:
                dependent = self.dict_to_array(dependent)
                
            page = merge_arrays([independents, dependent], flatten=True, usemask=False)
            self._pages[name] = self.sane_append(self._pages[name], page)
    
    def dataset(self, dset): 
        
        def value_parser(array):
            if len(array.shape) == 1: 
                return array.reshape((-1, 1))
            return array
        
        independents = [(name, value) for name, value in dset.items() if value.get('independent_parameter', False)]
        independents = self.dict_to_array(dict(independents), flatten=True, value_parser=value_parser)
        
        dependents = [(name, value) for name, value in dset.items() if not value.get('independent_parameter', False)]
        
        for name, parameter_record in dependents: 
            dependent = self.dict_to_array({name: parameter_record}, flatten=True, value_parser=value_parser)
            
            page = merge_arrays([independents, dependent], flatten=True, usemask=False)
            self._pages[name] = self.sane_append(self._pages[name], page)
    
    def __getitem__(self, item): 
        return self._pages.__getitem__(item)

In [5]:
store = NpStore()

In [6]:
store.add({
    "x": {"unit": "V", "value": 4.1, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 6.7, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": 0.7}
})



store.add({
    "x": {"unit": "V", "value": 3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 8.1, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": 0.9}
})

In [7]:
pprint(store["a"]["x [V]"])

array([[ 4.1],
       [ 3.5]])


In [8]:
pprint(store["a"]["y [V]"])

array([[ 6.7],
       [ 8.1]])


In [9]:
pprint(store["a"]["a [I]"])

array([[ 0.7],
       [ 0.9]])


In [10]:
store = NpStore()

In [11]:
store.add({
    "x": {"unit": "V", "value": 4.1, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 6.7, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": [0.2, -1.3]}
})



store.add({
    "x": {"unit": "V", "value": 3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 8.1, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": [4.5, -1.2]}
})

In [12]:
pprint(store["a"]["a [I]"])

array([[ 0.2, -1.3],
       [ 4.5, -1.2]])


In [13]:
store = NpStore()

In [14]:
store.add({
    "x": {"unit": "V", "value": 4.1, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 6.7, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": Delayed()}
})

store.add({
    "x": {"unit": "V", "value": 3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 8.0, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": Delayed()}
})

store.add({
    "x": {"unit": "V", "value": 3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 8.1, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": DelayedArray([[4.5, 3.4E-2], [-1.2, 9.0E-3], [-6.2, 2.0E-3]])}
})

store.add({
    "x": {"unit": "V", "value": -4.1, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": -6.7, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": Delayed()}
})

store.add({
    "x": {"unit": "V", "value": -3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": -8.0, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": Delayed()}
})

store.add({
    "x": {"unit": "V", "value": -3.5, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": -8.1, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": DelayedArray([[-4.5, -3.4E-2], [1.2, -9.0E-3], [6.2, 2.0E-3]])}
})

In [15]:
store["a"]["a [I]"]

array([[  4.50000000e+00,   3.40000000e-02],
       [ -1.20000000e+00,   9.00000000e-03],
       [ -6.20000000e+00,   2.00000000e-03],
       [ -4.50000000e+00,  -3.40000000e-02],
       [  1.20000000e+00,  -9.00000000e-03],
       [  6.20000000e+00,   2.00000000e-03]])

In [16]:
store["a"]["x [V]"]

array([[ 4.1],
       [ 3.5],
       [ 3.5],
       [-4.1],
       [-3.5],
       [-3.5]])

In [17]:
store["a"]["y [V]"]

array([[ 6.7],
       [ 8. ],
       [ 8.1],
       [-6.7],
       [-8. ],
       [-8.1]])

In [18]:
store = NpStore()

In [19]:
store.dataset({
    "x": {"unit": "V", "value": np.arange(100), 'independent_parameter': True}, 
    "y": {"unit": "V", "value": np.arange(100), 'independent_parameter': True}, 
    "a": {"unit": "I", "value": np.linspace(-1, 1, 100)}, 
    "b": {"unit": "I", "value": np.linspace(-3, 2, 100)}, 
})

In [20]:
store["a"]["a [I]"]

array([[-1.        ],
       [-0.97979798],
       [-0.95959596],
       [-0.93939394],
       [-0.91919192],
       [-0.8989899 ],
       [-0.87878788],
       [-0.85858586],
       [-0.83838384],
       [-0.81818182],
       [-0.7979798 ],
       [-0.77777778],
       [-0.75757576],
       [-0.73737374],
       [-0.71717172],
       [-0.6969697 ],
       [-0.67676768],
       [-0.65656566],
       [-0.63636364],
       [-0.61616162],
       [-0.5959596 ],
       [-0.57575758],
       [-0.55555556],
       [-0.53535354],
       [-0.51515152],
       [-0.49494949],
       [-0.47474747],
       [-0.45454545],
       [-0.43434343],
       [-0.41414141],
       [-0.39393939],
       [-0.37373737],
       [-0.35353535],
       [-0.33333333],
       [-0.31313131],
       [-0.29292929],
       [-0.27272727],
       [-0.25252525],
       [-0.23232323],
       [-0.21212121],
       [-0.19191919],
       [-0.17171717],
       [-0.15151515],
       [-0.13131313],
       [-0.11111111],
       [-0

In [21]:
store["a"]["x [V]"]

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40],
       [41],
       [42],
       [43],
       [44],
       [45],
       [46],
       [47],
       [48],
       [49],
       [50],
       [51],
       [52],
       [53],
       [54],
       [55],
       [56],
       [57],
       [58],
       [59],
       [60],
       [61],
       [62],
       [63],
       [64],
       [65],
       [66],
       [67],
       [68],
       [69],
       [70],
       [71],
       [72],
       [73],
       [74],
       [75],
       [76],

In [22]:
store["a"]["y [V]"]

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40],
       [41],
       [42],
       [43],
       [44],
       [45],
       [46],
       [47],
       [48],
       [49],
       [50],
       [51],
       [52],
       [53],
       [54],
       [55],
       [56],
       [57],
       [58],
       [59],
       [60],
       [61],
       [62],
       [63],
       [64],
       [65],
       [66],
       [67],
       [68],
       [69],
       [70],
       [71],
       [72],
       [73],
       [74],
       [75],
       [76],

In [23]:
store.add({
    "x": {"unit": "V", "value": 100, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 100, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": 1E-4}, 
    "b": {"unit": "I", "value": 2E-4}, 
})

In [24]:
store["a"]["a [I]"][-1]

array([ 0.0001])

In [25]:
store["a"]["x [V]"][-1]

array([100])

In [26]:
store = NpStore()

In [27]:
store.dataset({
    "x": {"unit": "V", "value": np.arange(100), 'independent_parameter': True}, 
    "y": {"unit": "V", "value": np.arange(100), 'independent_parameter': True}, 
    "a": {"unit": "I", "value": np.linspace(-1, 1, 200).reshape((100, -1))}, 
    "b": {"unit": "I", "value": np.linspace(-3, 2, 200).reshape((100, -1))}, 
})

In [28]:
store["a"]["a [I]"]

array([[-1.        , -0.98994975],
       [-0.9798995 , -0.96984925],
       [-0.95979899, -0.94974874],
       [-0.93969849, -0.92964824],
       [-0.91959799, -0.90954774],
       [-0.89949749, -0.88944724],
       [-0.87939698, -0.86934673],
       [-0.85929648, -0.84924623],
       [-0.83919598, -0.82914573],
       [-0.81909548, -0.80904523],
       [-0.79899497, -0.78894472],
       [-0.77889447, -0.76884422],
       [-0.75879397, -0.74874372],
       [-0.73869347, -0.72864322],
       [-0.71859296, -0.70854271],
       [-0.69849246, -0.68844221],
       [-0.67839196, -0.66834171],
       [-0.65829146, -0.64824121],
       [-0.63819095, -0.6281407 ],
       [-0.61809045, -0.6080402 ],
       [-0.59798995, -0.5879397 ],
       [-0.57788945, -0.5678392 ],
       [-0.55778894, -0.54773869],
       [-0.53768844, -0.52763819],
       [-0.51758794, -0.50753769],
       [-0.49748744, -0.48743719],
       [-0.47738693, -0.46733668],
       [-0.45728643, -0.44723618],
       [-0.43718593,

In [29]:
store.add({
    "x": {"unit": "V", "value": 100, 'independent_parameter': True}, 
    "y": {"unit": "V", "value": 100, 'independent_parameter': True}, 
    "a": {"unit": "I", "value": np.array([2.0, 3.0])}, 
})

In [30]:
store["a"]["a [I]"][-1]

array([ 2.,  3.])